In [6]:
import numpy as np
import cv2
import scipy.signal as sig

number_leds = 15
dist = 4 # in mm, distance between LEDs
size_led = 3.0
dist_from_center = 17.0

## ANGLE MADE BY LEDs WITH CENTER
theta1 = (size_led/dist_from_center)*(180.0/np.pi)
print('theta1',theta1)
## NUMBER OF ITERATIONS 
number_iterations = int(360/theta1)
number = int(np.log2(number_iterations))
number_iterations = np.power(2,number)
print('number_iterations', number_iterations)


dtheta = 360/number_iterations
print('dtheta', dtheta)

## IF TOTAL LENGTH FROM CENTER TO LAST LED IS 'k' mm, image should be k*k pixels.
imgs = cv2.imread('CS1.png',cv2.IMREAD_GRAYSCALE)
size = 2*(int(dist_from_center) + 14*dist +2)
img = cv2.resize(imgs, (size, size))
ret,img_th = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#cv2.imwrite(r'E:\Admin\Desktop\image1.png', img_th)
img_var = img_th
dsize = img_th.shape

center1 = int(len(img_th[0])/2)
print('center', center1)
#center = cv2.Point(center1, center1); #Assuming image is a square

bitstream = np.zeros((number_iterations,number_leds), dtype = int)
#print('shape', bitstream.shape)
ii=0;
for theta in np.arange(0,360,dtheta):    
    ii = ii+1
    M = cv2.getRotationMatrix2D((center1,center1), dtheta*theta, 1)
    img_var = cv2.warpAffine(img_th, M, dsize, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT);
    for n in np.arange(0,15):
        #print('img_var',img_var[center1 + n*dist,0])
        bitstream[int(theta/dtheta),n] = img_var[center1 + n*dist,center1]
        #print(img_var[center1 + n*dist,0])

m = np.array(([0,1,0],[1,4,1],[0,1,0]))*(1/8.0)
bitstream_th1 = sig.correlate2d(bitstream, m, mode = 'same', boundary = 'fill' ,fillvalue=0)
bitstream_th = bitstream_th1.astype(int)
#bitstream_final = np.where(bitstream_th)
#print('bitstream',bitstream_th)

thresh = 128
bitstream_th[bitstream_th<=thresh]=1
bitstream_th[bitstream_th>thresh]=0
bitstream_th = bitstream_th.astype(int)

#print('bitstream - final')
#for i in np.arange(0,len(bitstream_th)):
    #print(bitstream_th[i])
    
bitstream[bitstream<=thresh]=0
"""
print('bitstream - final')
for i in np.arange(0,len(bitstream)):
    print(bitstream[i])
"""    
bitstream[bitstream>thresh]=1

bits1 = np.zeros(len(bitstream_th), dtype = np.int)
bits2 = np.zeros(len(bitstream_th), dtype = np.int)
#print('bitstream - final')
t=0


for i in np.arange(0,len(bitstream_th)):
    #print(bitstream_th[i])
    for j in np.arange(0,8):
        bits1[i] = bits1[i]+ np.power(2,j)*bitstream_th[i][j]
    #bits1[i] = hex(bits1[i])
    #print(hex(bits1[i]))
    for j in np.arange(0,7):
        t=j
        if(j>=4):
            t=t+1
        bits2[i] = bits2[i] + np.power(2,t)*bitstream_th[i][j+8]
    #bits2[i] = hex(bits2[i])
    #print(hex(bits2[i]))
#print('bits1', bits1)
#print('bits2', bits2)
np.savetxt('bits11.txt', bits1, fmt = '%d', newline = ',')
np.savetxt('bits21.txt', bits2, fmt = '%d', newline = ',') 
#np.savetxt(r'E:\Admin\Desktop\bitstream.txt', bitstream_th)

#print(cv2.__version__)

('theta1', 10.111019914073351)
('number_iterations', 32)
('dtheta', 11)
('center', 75)


IndexError: index 32 is out of bounds for axis 0 with size 32